In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 66.7MB/s 
     |████████████████████████████████| 92kB 12.4MB/s 
     |████████████████████████████████| 3.8MB 23.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
import tensorflow as tf
import random
from konlpy.tag import Okt #openkorean
#konlpy : 한국어처리, okt는 트위터에서 만든 형태소 분석 패키지

In [3]:
EPOCHS = 200
NUM_WORDS = 2000

#Encoder

In [4]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder,self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS,64)
        self.lstm = tf.keras.layers.LSTM(512,return_sequences=True,return_state = True) #return state가 들어가야 hidden과 cell state
        #return_sequence = True를해서 모든타임스텝에대한 hidden
    def __call__(self, x, training = False, maks = None):
        x = self.emb(x)
        H, h, c = self.lstm(x)
        return H,h,c

#Decoder

In [5]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder,self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS,64)
        self.lstm = tf.keras.layers.LSTM(512,return_sequences=True,return_state=True)#return_sequence가 true여서 dense에 들어가는게 문장 전체로 들어가게 된다.
        #attension해서 concat
        self.att = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')
    
    def __call__(self,inputs, training =False,mask =None):
        y_,s0,c0,H=inputs#shifted output, hidden, cellstate 처음들어오는거, encoder단에서 새로오는거
        y_ = self.emb(y_)
        S,h,c = self.lstm(y_,initial_state=[s0,c0])#맨처음에 encoder에서 나온걸 넣어주고 나머진 shift

        #H는 key,value, S는 query(한 타임스텝앞의것)
        S_ = tf.concat([s0[:,tf.newaxis,:],S[:,:-1,:]],axis = 1)#2차원짜리 s0를 확장
        A = self.att([S_,H])#query,(keyvalue)
        y = tf.concat([S,A],axis = -1)

        return self.dense(y), h, c

#Seq2seq 모델 정의


In [6]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos#start of sequence
        self.eos = eos#end of sequence

    def __call__(self, inputs,training = False, maks = None):
        #autograph
        if training is True:
            x, y =inputs #output을 dec에 shifted 출력을 넣어야 하니까.
            H,h, c = self.enc(x) #hidden 과 cell state를 저장
            y, _, _ = self.dec((y,h,c,H))
            return y #y는 decorder결과라 전체 문장이 된다.

        #shifted 된걸 training에선 feeding하기 때문에 더 쉬운데
        #test땐 마지막 출력을 feeding해야하기 때문에 좀 더 복잡해진다.
        else:
            x = inputs
            H,h, c = self.enc(x) #context계산까진 똑같다.
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y,(1,1))

            seq = tf.TensorArray(tf.int32, 64)#최대 64의 길이로
            #autograph가 됨 tf for문이됨 다 tf로 쓰면
            for idx in tf.range(64):
                y, h, c = self.dec([y,h,c,H])# h,c는 encorder에서 받아온 hidden, cell state다. for 문 을 통해 이전 타임스텝의 값을 shift해서 얻게 된다.
                y = tf.cast(tf.argmax(y, axis = -1),dtype=tf.int32)#one-hot vector로 바꿔준다.sparse하게
                y = tf.reshape(y,(1,1)) #dimension이 하나가 되는데, batch하나의 값이라는 걸 표현하기 위해 고쳐준다.
                seq = seq.write(idx,y)

                if y == self.eos:
                    break
            return tf.reshape(seq.stack(),(1,64))#seq에 만들어둔걸 쌓아두는 작업이다.그리고 (배치,문장 최대길이)로 리쉐이프해서 출력해준다.

#루프 정의


In [7]:
@tf.function
def train_step(model,inputs,labels,loss_object,optimizer,train_loss,train_accuracy):
    output_labels = labels[:,1:]#맨앞은 sos라 뺀게 포함이 되고,
    shifted_labels = labels[:,:-1]#학습시 input으로 같이 넣어준다. seq2seq의 y이다. sos포함 eos빠짐. 즉, 오른쪽으로 shift된거
    with tf.GradientTape() as tape:
        predictions = model([inputs,shifted_labels],training = True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels,predictions)
  
@tf.function
def test_step(model,inputs):
    return model(inputs,training = False)#배치하나만 받아서 시퀀스 출력해준다!

#데이터셋

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
dataset_file = '/content/gdrive/My Drive/Colab Notebooks/aop-딥러닝-인공지능-강의자료/chatbot_data.csv'
okt = Okt()
#질문, 답, 질문, 답으로 이루어짐 카페에서의 문답!
with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line))for line in lines]#morphs는 한 줄 한줄 형태소 분석해주는거이다.
    #tokenizer에서 space를 기준으로 하기 때문에 ' '에 join을 해서 넣어준다.

questions = seq[::2]#홀수행
answers = ['\t'+lines for lines in seq[1::2]]#첫번째부터 짝수행을 가져온다. 탭의 역할은 start of sequence그리고 \n은 eos

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()

test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:#80%training, 
        train_q.append(q)
        train_a.append(a)

    else:#20% test
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')#빈도수가 높은 num_words개수로 토크나이즈

tokenizer.fit_on_texts(train_q+train_a)


#sparse 인코딩된 단어형태
train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',#결과니까 뒤에다가 0으로 padding한다.
                                                        maxlen=65)#맨뒤의 eos는 떼고 사용하니까
x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(1).prefetch(1024)

#학습 환경 정의

In [10]:
model = Seq2seq(sos = tokenizer.word_index['\t'],#sos
                eos = tokenizer.word_index['\n'])#eos

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [11]:
for epoch in range(EPOCHS):
  for seqs,labels in train_ds:
    train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

  template = 'Epoch:{}, Loss: {}, Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100))
  train_loss.reset_states()
  train_accuracy.reset_states()

Epoch:1, Loss: 2.6630282402038574, Accuracy: 84.4650650024414
Epoch:2, Loss: 0.5206864476203918, Accuracy: 91.78023529052734
Epoch:3, Loss: 0.48880404233932495, Accuracy: 92.01127624511719
Epoch:4, Loss: 0.4724532663822174, Accuracy: 92.15617370605469
Epoch:5, Loss: 0.4771592915058136, Accuracy: 92.0504379272461
Epoch:6, Loss: 0.46357032656669617, Accuracy: 92.16008758544922
Epoch:7, Loss: 0.45518746972084045, Accuracy: 92.1483383178711
Epoch:8, Loss: 0.44401174783706665, Accuracy: 92.25407409667969
Epoch:9, Loss: 0.4339218735694885, Accuracy: 92.285400390625
Epoch:10, Loss: 0.40951743721961975, Accuracy: 92.45378875732422
Epoch:11, Loss: 0.40366777777671814, Accuracy: 92.94329071044922
Epoch:12, Loss: 0.3896530866622925, Accuracy: 93.19392395019531
Epoch:13, Loss: 0.3838551640510559, Accuracy: 93.33098602294922
Epoch:14, Loss: 0.3718944787979126, Accuracy: 93.42105102539062
Epoch:15, Loss: 0.36484861373901367, Accuracy: 93.43672180175781
Epoch:16, Loss: 0.3587966561317444, Accuracy: 9

In [12]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q:',test_text)#input
    print('a:',gt_text)#정답
    print('p:',texts)#예측한 답변

_
q: ['아이스 아메리카노 하나요 \n']
a: ['\t테이크아웃 하실 건가 요 \n']
p: ['네 저희 가게 마스코트 메뉴 에요 \n']
_
q: ['저 카푸치노 로 주문 할게요 \n']
a: ['\t시럽 은 얼마나 뿌려 드릴 까요 \n']
p: ['네 알겠습니다 \n']
_
q: ['저 도장 다 모았는데 나중 에 써도 되나요 \n']
a: ['\t네 다음 에 써도 됩니다 \n']
p: ['쪽 한 사이즈 로만 판매 하고 있습니다 \n']
_
q: ['이 기프티콘 여기 서 사용 할 수 있나요 \n']
a: ['\t사용 가능하십니다 \n']
p: ['네 같이 적립 해 드리겠습니다 \n']
_
q: ['자바 칩 프라푸치노 에 휘핑 빼고요 \n']
a: ['\t6600원 입니다 \n']
p: ['아뇨 매장 에서는 로만 있습니다 \n']
_
q: ['따뜻한 아메리카노 한 잔 아이스 라떼 한 잔이요 \n']
a: ['\t드시고 가시나요 \n']
p: ['네 뜨거운 음료 로 드릴 까요 \n']
_
q: ['네 스콘 도 3 개 같이 주세요 \n']
a: ['\t스콘 도 드시고 가시나요 \n']
p: ['네 더 필요한 거 없으신 가요 \n']
_
q: ['카푸치노 차갑게 되나요 \n']
a: ['\t가능합니다 \n']
p: ['아니요 한 사이즈 로만 판매 하고 있습니다 \n']
_
q: ['아이스 카푸치노 개인 컵 에 주실 수 있나요 \n']
a: ['\t개인 컵 사이즈 가 작아서 음료 가 다 안 들어가요 \n']
p: ['네 같이 적립 해 드리겠습니다 \n']
_
q: ['음료 와 같이 먹을 빵 추천 해주세요 \n']
a: ['\t호두 베이글 이 잘 어울려요 \n']
p: ['네 담아 드립니다 \n']
_
q: ['모바일 쿠폰 으로 결제 할 수 있어요 \n']
a: ['\t네 가능해요 \n']
p: ['네 같이 적립 해 드리겠습니다 \n']
_
q: ['화장실 은 어디 에 있나요 \n']
a: ['\t오른쪽 벽쪽 으로 있어요 \n']
p: ['네 500원 할인 하시면